In [3]:
!pip install spire Spire.Xls openpyxl Unidecode -q

In [7]:
import openpyxl
import string
from unidecode import unidecode
from spire.xls import *
from spire.xls.common import *

file = "Excel\[1-006] Quy chuẩn ban hành văn bản nội bộ.xlsx"
# Function to remove punctuation from a text
def remove_punctuation(text):
    if isinstance(text, str):
        text = unidecode(text)
        return text.translate(str.maketrans('', '', string.punctuation))
    return text

# Load the existing workbook
wb = openpyxl.load_workbook(file)

# Get the active sheet or specify the sheet name
sheet = wb.active  # or wb['SheetName']

# Iterate through all cells in the sheet
for row in sheet.iter_rows():
    for cell in row:
        if cell.value:  # If the cell has a value (not empty)
            cell.value = remove_punctuation(cell.value)

base_name = os.path.splitext(file)[0]  # Lấy tên file gốc không có phần mở rộng
modified_file = f"{base_name}_format.xlsx"

# Save the modified workbook
wb.save(modified_file)

# Create a Workbook object
workbook = Workbook()
# Load an Excel file
workbook.LoadFromFile(modified_file)

for index, sheet in enumerate(workbook.Worksheets):
    # # Get the first worksheet
    # sheet = workbook.Worksheets[0]
    # Save the worksheet to an image
    image = sheet.ToImage(sheet.FirstRow, sheet.FirstColumn, sheet.LastRow, sheet.LastColumn)
    # Save the image to a PNG file
    image_file = f"{base_name}_sheet{index + 1}.png"
    image.Save(image_file)

workbook.Dispose()
print("Excel file processed successfully!")

Excel file processed successfully!


In [14]:
import os
import string
from unidecode import unidecode
from openpyxl import Workbook, load_workbook
from spire.xls import Workbook as SpireWorkbook
def remove_punctuation(text):
    if isinstance(text, str):
        text = unidecode(text)
        return text.translate(str.maketrans('', '', string.punctuation))
    return text

def save_sheets_as_images(file_path, output_folder):
    base_name = os.path.splitext(os.path.basename(file_path))[0]  # Lấy tên file gốc không có phần mở rộng
    modified_file = os.path.join(output_folder, f"{base_name}_format.xlsx")
    
    # Load and process the Excel file
    wb = load_workbook(file_path)
    sheet = wb.active  # or specify the sheet name
    
    # Iterate through all cells and remove punctuation
    for row in sheet.iter_rows():
        for cell in row:
            if cell.value:
                cell.value = remove_punctuation(cell.value)
    
    wb.save(modified_file)
    
    # Dùng Spire.XLS để load file đã chỉnh sửa
    workbook = SpireWorkbook()
    workbook.LoadFromFile(modified_file)
    
    # Duyệt qua từng sheet và lưu ảnh
    for index, sheet in enumerate(workbook.Worksheets):
        image = sheet.ToImage(sheet.FirstRow, sheet.FirstColumn, sheet.LastRow, sheet.LastColumn)
        image_file = os.path.join(output_folder, f"{base_name}_sheet{index + 1}.png")
        image.Save(image_file)
    
    workbook.Dispose()
    print(f"All sheets from {file_path} have been processed and saved as images in {output_folder}.")

def process_folder(folder_path):
    output_folder = os.path.join(folder_path, "image_excel")
    os.makedirs(output_folder, exist_ok=True)
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".xlsx"):
            file_path = os.path.join(folder_path, file_name)
            save_sheets_as_images(file_path, output_folder)

# Example usage
process_folder("Excel_Sample/")

All sheets from Excel_Sample/[ISMS_1-501_VN] 1. Các quy định An ninh thông tin .xlsx have been processed and saved as images in Excel_Sample/image_excel.


In [25]:
import os
import string
from unidecode import unidecode
from openpyxl import Workbook, load_workbook
from spire.xls import Workbook as SpireWorkbook

def remove_punctuation(text):
    if isinstance(text, str):
        text = unidecode(text)
        return text.translate(str.maketrans('', '', string.punctuation))
    return text

def get_actual_used_range(sheet):
    max_rows = sheet.Rows.Length
    max_cols = sheet.Columns.Length

    for row in range(max_rows - 1, -1, -1):
        for col in range(max_cols):
            cell = sheet.Range[row + 1, col + 1]
            if cell.Value is not None and cell.Text.strip():
                last_row = row
                break

    for col in range(max_cols - 1, -1, -1):
        for row in range(max_rows):
            cell = sheet.Range[row + 1, col + 1]
            if cell.Value is not None and cell.Text.strip():
                last_col = col
                break
    
    first_row = sheet.FirstRow
    first_col = sheet.FirstColumn
    
    return first_row, first_col, last_row, last_col

def save_sheets_as_images(file_path, output_folder):
    print(f"Start convert from {file_path} have been processed.")
    base_name = os.path.splitext(os.path.basename(file_path))[0]  # Lấy tên file gốc không có phần mở rộng
    modified_file = os.path.join(output_folder, f"{base_name}_format.xlsx")
    
    # Load and process the Excel file
    wb = load_workbook(file_path)
    sheet = wb.active  # or specify the sheet name
    
    # Iterate through all cells and remove punctuation
    for row in sheet.iter_rows():
        for cell in row:
            if cell.value:
                cell.value = remove_punctuation(cell.value)
    
    wb.save(modified_file)
    
    # Dùng Spire.XLS để load file đã chỉnh sửa
    workbook = SpireWorkbook()
    workbook.LoadFromFile(modified_file)
    
    # Duyệt qua từng sheet và lưu ảnh
    for index, sheet in enumerate(workbook.Worksheets):
        first_row, first_col, last_row, last_col = get_actual_used_range(sheet)
        # print(f"Sheet {index}: FirstRow={first_row}, LastRow={last_row}, FirstColumn={first_col}, LastColumn={last_col}")
        if first_row <= last_row and first_col <= last_col:
            image = sheet.ToImage(first_row, first_col, last_row, last_col)
            image_file = os.path.join(output_folder, f"{base_name}_sheet{index + 1}.png")
            image.Save(image_file)
    
    workbook.Dispose()
    print(f"All sheets from {file_path} have been processed and saved as images in {output_folder}.")

def process_folder(folder_path):
    output_folder = os.path.join(folder_path, "image_excel")
    os.makedirs(output_folder, exist_ok=True)
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".xlsx"):
            file_path = os.path.join(folder_path, file_name)
            save_sheets_as_images(file_path, output_folder)

# Example usage
process_folder("Excel_v2/")

Start convert from Excel_v2/0817【日経R回答】02_別紙２_(3)【顧】個人番号・個人データ保護に関する対応状況のチェックリスト.xlsx have been processed.
All sheets from Excel_v2/0817【日経R回答】02_別紙２_(3)【顧】個人番号・個人データ保護に関する対応状況のチェックリスト.xlsx have been processed and saved as images in Excel_v2/image_excel.
Start convert from Excel_v2/0825【日経R回答】01_別紙１_①【顧】受託業務に関するチェックリスト.xlsx have been processed.
All sheets from Excel_v2/0825【日経R回答】01_別紙１_①【顧】受託業務に関するチェックリスト.xlsx have been processed and saved as images in Excel_v2/image_excel.
Start convert from Excel_v2/1226NKR_情報セキュリティチェック表.xlsx have been processed.
All sheets from Excel_v2/1226NKR_情報セキュリティチェック表.xlsx have been processed and saved as images in Excel_v2/image_excel.
Start convert from Excel_v2/2-002_QUY TRÌNH QUẢN LÝ VÀ YÊU CẦU CON DẤU CÔNG TY.xlsx have been processed.
All sheets from Excel_v2/2-002_QUY TRÌNH QUẢN LÝ VÀ YÊU CẦU CON DẤU CÔNG TY.xlsx have been processed and saved as images in Excel_v2/image_excel.
Start convert from Excel_v2/2-109_QUY TRÌNH CẬP NHẬT QUY CHẾ TÀI CHÍNH.xlsx 

c:\Users\Admin\anaconda3\envs\learn\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


All sheets from Excel_v2/Literature Review.xlsx have been processed and saved as images in Excel_v2/image_excel.
Start convert from Excel_v2/money-manager-2.xlsx have been processed.


c:\Users\Admin\anaconda3\envs\learn\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
c:\Users\Admin\anaconda3\envs\learn\lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: Goals!$A:$I.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


All sheets from Excel_v2/money-manager-2.xlsx have been processed and saved as images in Excel_v2/image_excel.
Start convert from Excel_v2/project-manager-gantt-chart.xlsx have been processed.
All sheets from Excel_v2/project-manager-gantt-chart.xlsx have been processed and saved as images in Excel_v2/image_excel.
Start convert from Excel_v2/sales-forecast.xlsx have been processed.


UnboundLocalError: local variable 'last_row' referenced before assignment

In [31]:
import openpyxl


def autofit_excel(file_path, output_path):
    wb = openpyxl.load_workbook(file_path)

    for ws in wb.worksheets:  # Iterate through all sheets
        column_widths = {}  # Store maximum column width

        for row in ws.iter_rows():
            max_height = 0  # Store maximum row height

            for cell in row:
                if cell.value:
                    cell_text = str(cell.value)

                    # Adjust column width based on the longest content
                    col_letter = cell.column_letter  # Get column letter (A, B, C, ...)
                    column_widths[col_letter] = max(column_widths.get(col_letter, 0), len(cell_text) + 2)

                    # Calculate row height based on number of lines
                    num_lines = cell_text.count("\n") + 1  # Count number of lines
                    max_height = max(max_height, num_lines * 20)  # Adjust factor (20px per line)

            # Update row height
            ws.row_dimensions[row[0].row].height = max_height if max_height > 0 else None

        # Update column width
        for col_letter, width in column_widths.items():
            ws.column_dimensions[col_letter].width = width

    # Save the modified file
    wb.save(output_path)
    print(f"File has been saved at: {output_path}")

# Run the function with the Excel file path
autofit_excel("Excel_v2\[1-006] Quy chuẩn ban hành văn bản nội bộ.xlsx", "refine_excel.xlsx")


File has been saved at: refine_excel.xlsx
